In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
from sksurv.util import Surv 
from sklearn.model_selection import train_test_split

In [7]:
import yaml
import sys
sys.path.append('/home/gmarinos/Documents/Code/threshold_exceedance_forecasting/src/')
 
from synthetic_data_generation.synthetic_data_generation import *
from utils.utils import *
from data_fetching.data_fetching import *
from dataset_construction.dataset_construction import *
from main_classes.main_classes import *

data = generate_simulated_multivariate_time_series(n_steps=1000, n_series=1).rename(columns={'Unnamed: 0':'Date', 'Series 1':'Incidents'})
# data = data.drop(columns=['Series 2', 'Series 3'])
data.head()

,Incidents
2022-01-31,NaN
2022-02-28,NaN
2022-03-31,NaN
2022-04-30,NaN
2022-05-31,NaN


In [3]:
# Step 1: Parse the YAML file
# Define the path to your config file
config_file_path = '/home/gmarinos/Documents/Code/threshold_exceedance_forecasting/config.yaml'

# Load the config file
with open(config_file_path, 'r') as config_file:
    config_data = yaml.safe_load(config_file)


###Data API

apikey = config_data['apikey']

headers = config_data['headers']

search_endpoint = config_data['search_endpoint']

data_dir = config_data['data_dir']

    
###CONFIGURATION

data_ingredient = config_data['data_requests']['data_ingredient']

predictions_ingredient = config_data['data_requests']['predictions_ingredient']

data_hazard = config_data['data_requests']['data_hazard']

predictions_hazard = config_data['data_requests']['predictions_hazard']



#####

THRESHOLD_PERCENTILE = config_data['THRESHOLD_PERCENTILE']

timesteps = config_data['timesteps']

random_state = config_data['random_state']

Column_name = config_data['Column_name']

test_size_ = config_data['test_size_']

shuffle_ = config_data['shuffle_']

invalid_observations_removal_ = config_data['invalid_observations_removal']

interval = config_data['interval'] 

starting_date = config_data['starting_date']

end_date = config_data['end_date']

validation_rows = len(data_prediction_request(apikey, headers, search_endpoint, data_dir, predictions_ingredient, predictions_hazard, interval, starting_date, end_date))

In [ ]:
from main_classes.main_classes.data_preprocessing import DataPreprocessor 
from main_classes.main_classes.model_trainning_and_evaluation import ModelEvaluator

# Create an instance of the DataPreprocessing class
data_processor = DataPreprocessor(data, apikey, headers, search_endpoint,
                 data_dir, data_ingredient, predictions_ingredient, data_hazard, predictions_hazard,
                 THRESHOLD_PERCENTILE, timesteps, random_state, Column_name, test_size_, shuffle_,
                 invalid_observations_removal_, interval, starting_date, end_date)

# Step 1: Data Preprocessing
X, y = data_processor.create_supervised_data()

# Step 2: Train-Test Split
test_size_ = 0.2  # Set your test size
X_train, X_test, y_train, y_test, THRESHOLD, X_train_survival, X_test_survival, y_train_survival, y_test_survival, X_train_survival_ext_feat, X_test_survival_ext_feat, y_train_survival_ext_feat, y_test_survival_ext_feat = data_processor.train_test_split(test_size_)

# Step 3: Invalid Observations Removal (optional)
invalid_observations_removal_ = False  # Set to True if you want to remove invalid observations
X_train, X_test, y_train, y_test = data_processor.invalid_observations_removal(X_train, X_test, y_train, y_test, THRESHOLD, invalid_observations_removal_)

In [6]:
model_evaluator = ModelEvaluator(data, 
                                  X_train, X_test, y_train, y_test, THRESHOLD, 
                                  X_train_survival, X_test_survival, y_train_survival, y_test_survival, 
                                  X_train_survival_ext_feat, X_test_survival_ext_feat, y_train_survival_ext_feat, y_test_survival_ext_feat,
                                  Column_name, timesteps, THRESHOLD_PERCENTILE,
                                   validation_rows, random_state, test_size_, shuffle_,
                                   invalid_observations_removal_, Column_name, interval, data_ingredient, 
                                   predictions_ingredient, data_hazard, predictions_hazard)

In [7]:
table_data = model_evaluator.evaluate_models()
model_evaluator.print_results()



Distribution: genextreme
Model: DecisionTreeRegressor(random_state=33)
Confusion Matrix:
[[23  8]
 [18  6]]
Accuracy: 0.53
Precision: 0.43
Recall: 0.25
F1 Score: 0.32
Roc AUC: 0.67


Distribution: norm
Model: DecisionTreeRegressor(random_state=33)
Confusion Matrix:
[[28  3]
 [18  6]]
Accuracy: 0.62
Precision: 0.67
Recall: 0.25
F1 Score: 0.36
Roc AUC: 0.67


Distribution: laplace
Model: DecisionTreeRegressor(random_state=33)
Confusion Matrix:
[[28  3]
 [18  6]]
Accuracy: 0.62
Precision: 0.67
Recall: 0.25
F1 Score: 0.36
Roc AUC: 0.67


Distribution: logistic
Model: DecisionTreeRegressor(random_state=33)
Confusion Matrix:
[[28  3]
 [18  6]]
Accuracy: 0.62
Precision: 0.67
Recall: 0.25
F1 Score: 0.36
Roc AUC: 0.67


Distribution: gumbel_r
Model: DecisionTreeRegressor(random_state=33)
Confusion Matrix:
[[23  8]
 [18  6]]
Accuracy: 0.53
Precision: 0.43
Recall: 0.25
F1 Score: 0.32
Roc AUC: 0.67


Distribution: lognorm
Model: DecisionTreeRegressor(random_state=33)
Confusion Matrix:
[[28  3]
 

/home/gmarinos/Documents/Code/1st_publication/1st_pub/lib/python3.10/site-packages/scipy/stats/_continuous_distns.py:3263: RuntimeWarning: overflow encountered in exp
  return np.exp(-np.exp(-x))


Confusion Matrix:
[[19 12]
 [ 7 17]]
Accuracy: 0.65
Precision: 0.59
Recall: 0.71
F1 Score: 0.64
Roc AUC: 0.80


Distribution: norm
Model: AdaBoostRegressor(random_state=33)
Confusion Matrix:
[[30  1]
 [21  3]]
Accuracy: 0.60
Precision: 0.75
Recall: 0.12
F1 Score: 0.21
Roc AUC: 0.80


Distribution: laplace
Model: AdaBoostRegressor(random_state=33)
Confusion Matrix:
[[30  1]
 [21  3]]
Accuracy: 0.60
Precision: 0.75
Recall: 0.12
F1 Score: 0.21
Roc AUC: 0.80


Distribution: logistic
Model: AdaBoostRegressor(random_state=33)
Confusion Matrix:
[[30  1]
 [21  3]]
Accuracy: 0.60
Precision: 0.75
Recall: 0.12
F1 Score: 0.21
Roc AUC: 0.80


Distribution: gumbel_r
Model: AdaBoostRegressor(random_state=33)
Confusion Matrix:
[[19 12]
 [ 7 17]]
Accuracy: 0.65
Precision: 0.59
Recall: 0.71
F1 Score: 0.64
Roc AUC: 0.80


Distribution: lognorm
Model: AdaBoostRegressor(random_state=33)
Confusion Matrix:
[[30  1]
 [19  5]]
Accuracy: 0.64
Precision: 0.83
Recall: 0.21
F1 Score: 0.33
Roc AUC: 0.79


Distribut

/home/gmarinos/Documents/Code/1st_publication/1st_pub/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/gmarinos/Documents/Code/1st_publication/1st_pub/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/gmarinos/Documents/Code/1st_publication/1st_pub/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result)

Confusion Matrix:
[[31  0]
 [24  0]]
Accuracy: 0.56
Precision: 0.00
Recall: 0.00
F1 Score: 0.00
Roc AUC: 0.50


Distribution: logistic
Model: SVR(C=1, gamma=2)
Confusion Matrix:
[[31  0]
 [24  0]]
Accuracy: 0.56
Precision: 0.00
Recall: 0.00
F1 Score: 0.00
Roc AUC: 0.50


Distribution: gumbel_r
Model: SVR(C=1, gamma=2)
Confusion Matrix:
[[31  0]
 [24  0]]
Accuracy: 0.56
Precision: 0.00
Recall: 0.00
F1 Score: 0.00
Roc AUC: 0.50


Distribution: lognorm
Model: SVR(C=1, gamma=2)
Confusion Matrix:
[[31  0]
 [24  0]]
Accuracy: 0.56
Precision: 0.00
Recall: 0.00
F1 Score: 0.00
Roc AUC: 0.50


Distribution: cauchy
Model: SVR(C=1, gamma=2)
Confusion Matrix:
[[31  0]
 [24  0]]
Accuracy: 0.56
Precision: 0.00
Recall: 0.00
F1 Score: 0.00
Roc AUC: 0.50


Distribution: genextreme
Model: MLPRegressor(random_state=33)


...

/home/gmarinos/Documents/Code/1st_publication/1st_pub/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:684: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Confusion Matrix:
[[22  9]
 [ 8 16]]
Accuracy: 0.69
Precision: 0.64
Recall: 0.67
F1 Score: 0.65
Roc AUC: 0.78


Distribution: cauchy
Model: MLPRegressor(random_state=33)


/home/gmarinos/Documents/Code/1st_publication/1st_pub/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:684: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Confusion Matrix:
[[24  7]
 [ 8 16]]
Accuracy: 0.73
Precision: 0.70
Recall: 0.67
F1 Score: 0.68
Roc AUC: 0.79
0.6545454545454545 % 1.0 % 0.20833333333333334 % 0.3448275862068966 % 0.282258064516129 %
0.6727272727272727 % 1.0 % 0.25 % 0.4 % 0.282258064516129 %
0.6363636363636364 % 0.8333333333333334 % 0.20833333333333334 % 0.33333333333333337 % 0.2264784946236559 %
0.6727272727272727 % 0.8 % 0.3333333333333333 % 0.47058823529411764 % 0.2264784946236559 %
0.6363636363636364 % 0.8333333333333334 % 0.20833333333333334 % 0.33333333333333337 % 0.2264784946236559 %
0.6363636363636364 % 0.8333333333333334 % 0.20833333333333334 % 0.33333333333333337 % 0.2264784946236559 %
0.6363636363636364 % 0.8333333333333334 % 0.20833333333333334 % 0.33333333333333337 % 0.24731182795698925 %
0.6545454545454545 % 0.8571428571428571 % 0.25 % 0.3870967741935483 % 0.24731182795698925 %
0.6363636363636364 % 0.8333333333333334 % 0.20833333333333334 % 0.33333333333333337 % 0.2547043010752688 %
0.6545454545454545 % 

"| Data                                                                                                            |   Threshold |   Number of lags | Distribution   | Algorithm                                              | Accuracy   | Precision   | Recall   | F1-score   | Roc AUC   |   Test size | Shuffle   | Invalid Observations removal   |   No of Columns | Resolution   |\n|:----------------------------------------------------------------------------------------------------------------|------------:|-----------------:|:---------------|:-------------------------------------------------------|:-----------|:------------|:---------|:-----------|:----------|------------:|:----------|:-------------------------------|----------------:|:-------------|\n| data_ingredient:nuts, nut products and seeds _ predictions_ingredient:nuts, nut products and seeds _ _Incidents |          80 |                6 | genextreme     | DecisionTreeRegressor(random_state=33)                 | 53%        | 43%  

In [8]:
model_evaluator.get_results_df()